# Прогнозирование оттока клиентов телеком-оператора

Импорт библиотек

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Исследовательский анализ данных

### Обзор файлов с данными

In [14]:
try:
    df_contract = pd.read_csv('/home/mike/repos/contract.csv')
except FileNotFoundError:
    df_contract = pd.read_csv('/datasets/final_provider/contract.csv')
df_contract.head()

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65


In [10]:
try:
    df_internet = pd.read_csv('/home/mike/repos/internet.csv')
except FileNotFoundError:
    df_internet = pd.read_csv('/datasets/final_provider/internet.csv')
df_internet.head()

,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,Fiber optic,No,No,No,No,No,No


In [11]:
try:
    df_personal = pd.read_csv('/home/mike/repos/personal.csv')
except FileNotFoundError:
    df_personal = pd.read_csv('/datasets/final_provider/personal.csv')
df_personal.head()

,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No


In [12]:
try:
    df_phone = pd.read_csv('/home/mike/repos/phone.csv')
except FileNotFoundError:
    df_phone = pd.read_csv('/datasets/final_provider/phone.csv')
df_phone.head()

,customerID,MultipleLines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes


### Обзор данных и первичная предобработка

Изучение данных с помощью методов:
* df.info() - общее количество данных, проверка наименований столбцов (переименование при необходимости), проверка на соответствие типов данных (изменение типов данных при необходимости), наличие пропущенных значений (анализ возможных стратегий избавления от пропусков: удаление объектов с пропущенными значениями, заполнение нулями, заполнение медианой и т.п.).
* df.describe() - для числовых данных анализ рахмаха (обоснованность максимальных и минимальных значений и удаление объектов с ошибочными данными, квартильный анализ).
* s.plot(kind='kde') - построение графиков плотности распределения для цифровых данных, оценка формы распределения.
* s.value_counts() - для категориальных данных подсчет количества объектов в каждой категории (построение столбчатых диаграм для наглядности).

Дополнительно обзор данных отдельно для оставшихся клиентов и ушедших для оценки наиболее значимых факторов (датафреймы объединяются по полю `customerID`)

### Визуальный анализ данных

* Построение sns.pairplot() для оценки линейной корреляции
* Построение boxplot() для оценки количества выбросов (полтора межквартильных интервала)
* Построение матрицы корреляции числовых признаков для оценки наиболее значимых факторов и мультиколлинеарности

### Предобработка данных

* Удаление пропущенных значений (если в результате анализа принято решение удалять пропцщенные значения или их часть)
* Заполнение пропущенных значений в соответствии с выранной стратегией (не влекущей утечку данных)
* Кодирование целевого признака `EndDate` (No = 0, дата = 1)
* Кодирование категориальных признаков (с небольшим количеством значений - OneHotEncoding, с большим количеством значений - OrdinalEncoding). На этом этапе оставляем датафрей с некодированными категориальными данными, который будем использовать для работы с CatBoost.
* Разделение выборки на обучающую и тестовую (hold out)

## Построение моделей

### Построение моделей на исходном датасете

Построение LogisticRegression, kNN, RandomForest, LightGBM, CatBoost (на датасете без кодирования категориальных признаков). Выбор лучшей модели для дальнейшей тонкой настройки

### Перобразование существующих и создание дополнительных признаков

* Масштабирование числовых признаков
* Логарифмирование числовых признаков с распределением отличным от нормального
* Добавление столбцов для выбросов (полтора межквартильных интервала, три межквартильных интервала)
* Добавление признаков кластеризацией и поиском аномалий
* Определение наиболее важных признаков, перестроение моделей только на них, чтобы избавиться от возможного шума, снижение размерности (PCA)
* После определения наиболее точной модели и лучшего датасета для нее можно попробовать создать дополнительные признаки как предсказания более слабых моделей (bagging?)

### Тонкая настройка самой точной модели

Подбор гиперпараметров самой точной модели с помощью optuna